In [2]:
#Implementation using Langchain SQL Agent
# It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
# It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
# It can query the database as many times as needed to answer the user question.
# It will save tokens by only retrieving the schema from relevant tables.

from dotenv import load_dotenv
load_dotenv()

# Access environment variables
import os
openai_key = os.getenv("API_KEY")
db_url = os.getenv("DB_URL")

In [3]:
#Using opneai-tools Agent
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit
import psycopg2

In [4]:
# Setup PostgreSQL database using SQLDatabaseToolkit
db = SQLDatabase.from_uri(
    db_url
)
print(db.dialect)
print(db.get_usable_table_names())

postgresql
['ADM2022', 'C2022DEP', 'C2022_A', 'C2022_B', 'C2022_C', 'EFFY2022', 'EFFY2022_DIST', 'EFIA2022', 'FLAGS2022', 'GR200_22', 'GR2022', 'GR2022_L2', 'GR2022_PELL_SSL', 'HD2022', 'IC2022', 'IC2022_AY', 'IC2022_CAMPUSES', 'IC2022_PY', 'OM2022', 'SFA2122', 'SFAV2122']


In [5]:
#Initializing the Agent with create_sql_agent constructor
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [6]:
#Invoking the query with SQLAgent
#he agent will first choose which tables are relevant and then add the schema for those tables and a few sample rows to the prompt.
agent_executor.invoke(
    "How many rows are present in ADM2022"

)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


ADM2022, C2022DEP, C2022_A, C2022_B, C2022_C, EFFY2022, EFFY2022_DIST, EFIA2022, FLAGS2022, GR200_22, GR2022, GR2022_L2, GR2022_PELL_SSL, HD2022, IC2022, IC2022_AY, IC2022_CAMPUSES, IC2022_PY, OM2022, SFA2122, SFAV2122
Invoking: `sql_db_schema` with `ADM2022`



CREATE TABLE "ADM2022" (
	index BIGINT, 
	unitid BIGINT, 
	admcon1 BIGINT, 
	admcon2 BIGINT, 
	admcon3 BIGINT, 
	admcon4 BIGINT, 
	admcon5 BIGINT, 
	admcon6 BIGINT, 
	admcon7 BIGINT, 
	admcon8 BIGINT, 
	admcon9 BIGINT, 
	admcon10 BIGINT, 
	admcon11 BIGINT, 
	admcon12 BIGINT, 
	xapplcn TEXT, 
	applcn BIGINT, 
	xapplcnm TEXT, 
	applcnm BIGINT, 
	xapplcnw TEXT, 
	applcnw BIGINT, 
	xapplcnan TEXT, 
	applcnan DOUBLE PRECISION, 
	xapplcnun TEXT, 
	applcnun BIGINT, 
	xadmssn TEXT, 
	admssn DOUBLE PRECISION, 
	xadmssnm TEXT, 
	admssnm DOUBLE PRECISION, 
	xadmssnw TEXT, 
	admssnw DOUBLE PRECISION, 
	xadmssnan TEXT, 
	admssnan DOUBLE PRECISION, 
	xadmssnun 

{'input': 'How many rows are present in ADM2022',
 'output': 'There are 1,991 rows present in the ADM2022 table.'}

In [7]:
#Implementing dynamic few-shot prompt
examples = [
    {
        "input": "Institutes which require Secondary School GPA for getting admission in Undergrad program.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon1 = 1;",
    },
    {
        "input": "Institutes which do not require Secondary School GPA for getting admission in Undergrad program but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon1 = 5;",
    },
    {
        "input": "Institutes which do not require Secondary School GPA for getting admission in Undergrad program but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon1 = 3;",
    },
    {
        "input": "Institutes which require Secondary school rank for getting admission in Undergrad program.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon2 = 1;",
    },
    {
        "input": "Institutes which do not require Secondary school rank for getting admission in Undergrad program but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon2 = 5;",
    },
    {
        "input": "Institutes which do not require Secondary school rank for getting admission in undergrad but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon2 = 3;",
    },
    {
        "input": "Institutes which require Secondary school record for getting admission in Undergrad program.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon3 = 1;",
    },
    {
        "input": "Institutes which do not require Secondary school record for getting admission in Undergrad program but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon3 = 5;",
    },
    {
        "input": "Institutes which do not require Secondary school record for getting admission in Undergrad program but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon3 = 3;",
    },
    {
        "input": "Institutes which require Completion of college-preparatory program for getting admission in undergrad.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon4 = 1;",
    },
    {
        "input": "Institutes which do not require Completion of college-preparatory program for getting admission in undergrad but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon4 = 5;",
    },
    {
        "input": "Institutes which do not require Completion of college-preparatory program for getting admission in undergrad but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon4 = 3;",
    },
    {
        "input": "Institutes which require Recommendations for getting admission in undergrad.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon5 = 1;",
    },
    {
        "input": "Institutes which do not require Recommendations for getting admission in undergrad but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon5 = 5;",
    },
    {
        "input": "Institutes which do not require Recommendations for getting admission in undergrad but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon5 = 3;",
    },
        {
        "input": "Institutes which do not require Personal statement or essay for getting admission in undergrad but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon11 = 3;"
    },
    {
        "input": "Institutes which require Legacy status for getting admission in undergrad.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon12 = 1;"
    },
    {
        "input": "Institutes which do not require Legacy status for getting admission in undergrad but considered if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon12 = 5;"
    },
    {
        "input": "Institutes which do not require Legacy status for getting admission in undergrad but even if submitted.",
        "query": "SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon12 = 3;"
    },
    {
        "input": "Top 5 institutes in each state shows Highest number of applicants applied for the fall term undergrad admission.",
        "query": "SELECT DISTINCT IC.pccity, IC.index, SUM(ADM.applcn) AS applicant_count FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.applcn IS NOT NULL GROUP BY IC.pccity, IC.index ORDER BY SUM(ADM.applcn) DESC LIMIT 5;"
    },
    {
        "input": "Top 5 institutes in each state shows Highest number of men applicants applied for the fall term undergrad admission.",
        "query": "SELECT DISTINCT IC.pccity, IC.index, SUM(ADM.applcnm) AS applicant_count FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.applcnm IS NOT NULL GROUP BY IC.pccity, IC.index ORDER BY SUM(ADM.applcnm) DESC LIMIT 5;"
    },
    {
        "input": "Top 5 institutes in each state shows Highest number of women applicants applied for the fall term undergrad admission.",
        "query": "SELECT DISTINCT IC.pccity, IC.index, SUM(ADM.applcnw) AS applicant_count FROM public.\"ADM2022\" AS ADM INNER JOIN public.\"IC2022_CAMPUSES\" AS IC ON ADM.unitid = IC.index WHERE ADM.applcnw IS NOT NULL GROUP BY IC.pccity, IC.index ORDER BY SUM(ADM.applcnw) DESC LIMIT 5;"
    },
    
]


In [8]:
#using SemanticSimilarityExampleSelector, which will perform a semantic search using the embeddings and vector store we configure to find the examples most similar to our input
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [9]:
#Creating a few-shot prompt using FewShotPromptTemplate
#The few-shot prompt will be used for our system message
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a PostgreSQL database.
Given an input question, create a syntactically correct {dialect} PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [10]:
#Calling a full prompt with Human Message and agent_scratchpad MessagesPlaceholder
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [11]:
# Executing full prompt
prompt_val = full_prompt.invoke(
    {
        "input": " Institutes which require Work experience for getting admission in undergrad",
        "top_k": 5,
        "dialect": "PostgreSQL",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a PostgreSQL database.
Given an input question, create a syntactically correct PostgreSQL PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database,

In [12]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [13]:
agent.invoke({"input": " Institutes which require Work experience for getting admission in undergrad"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT IC.campusid, IC.pcaddr, IC.pccity FROM public."ADM2022" AS ADM INNER JOIN public."IC2022_CAMPUSES" AS IC ON ADM.unitid = IC.index WHERE ADM.admcon6 = 1;`


[('CUNY Brooklyn College - Feirstein Graduate School of Cinema', 'Brooklyn', 'NY'), ('Pratt Manhattan-A Division of Pratt Institute', 'New York', 'NY'), ('HCI College - Fort Lauderdale Campus', 'Fort Lauderdale', 'FL')]The institutes that require work experience for getting admission in undergrad are:
1. CUNY Brooklyn College - Feirstein Graduate School of Cinema in Brooklyn, NY
2. Pratt Manhattan-A Division of Pratt Institute in New York, NY
3. HCI College - Fort Lauderdale Campus in Fort Lauderdale, FL

> Finished chain.


{'input': ' Institutes which require Work experience for getting admission in undergrad',
 'output': 'The institutes that require work experience for getting admission in undergrad are:\n1. CUNY Brooklyn College - Feirstein Graduate School of Cinema in Brooklyn, NY\n2. Pratt Manhattan-A Division of Pratt Institute in New York, NY\n3. HCI College - Fort Lauderdale Campus in Fort Lauderdale, FL'}